# 向量检索构建提示词

In [1]:
from langchain_community.chat_models import ChatTongyi
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_community.embeddings import DashScopeEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [12]:
def print_prompt(prompt):
    print("=" * 20)
    print(prompt.to_string())
    print("=" * 20)
    return prompt

In [13]:
model = ChatTongyi(model="qwen3-max")
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "以我提供的已知参考资料为主，简洁和专业的回答用户问题。参考资料:{context}。"),
        ("user", "用户提问：{input}")
    ]
)

In [14]:
vector_store = InMemoryVectorStore(embedding=DashScopeEmbeddings(model="text-embedding-v4"))

In [15]:
# 准备一下资料（向量库的数据）
# add_texts 传入一个 list[str]
vector_store.add_texts(
    ["减肥就是要少吃多练", "在减脂期间吃东西很重要,清淡少油控制卡路里摄入并运动起来", "跑步是很好的运动哦"])

['573bb4ce-213a-47ac-95f3-4d8e97b4134e',
 '4581bee5-c535-4789-8267-611e1f2de132',
 'b2ecc877-5563-4620-bcfb-dc20660ad057']

In [16]:
input_text = "怎么减肥？"

In [17]:
# 检索向量库
result = vector_store.similarity_search(input_text, 2)
reference_text = "["
for doc in result:
    reference_text += doc.page_content
reference_text += "]"
print(result)

[Document(id='573bb4ce-213a-47ac-95f3-4d8e97b4134e', metadata={}, page_content='减肥就是要少吃多练'), Document(id='4581bee5-c535-4789-8267-611e1f2de132', metadata={}, page_content='在减脂期间吃东西很重要,清淡少油控制卡路里摄入并运动起来')]


In [18]:
chain = prompt | print_prompt | model | StrOutputParser()

In [19]:
res = chain.invoke({"input": input_text, "context": reference_text})
print(res)

System: 以我提供的已知参考资料为主，简洁和专业的回答用户问题。参考资料:[减肥就是要少吃多练在减脂期间吃东西很重要,清淡少油控制卡路里摄入并运动起来]。
Human: 用户提问：怎么减肥？
减肥的关键在于“少吃多练”：  
1. **饮食方面**：选择清淡、少油的食物，严格控制每日卡路里摄入；  
2. **运动方面**：坚持规律运动，增加热量消耗。  

通过合理饮食与科学锻炼相结合，才能健康有效地减脂。
